In [1]:
#Celdas de impotacion
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from ipywidgets import widgets
from datetime import datetime
from pymongo import MongoClient
import fastText
import cryptography
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import base64

In [2]:
client = MongoClient('localhost', 27017)

#Base de datos
db = client.FB_Scrap

#Obtenemos Coleciones
DB_post = db.FB_Posts
DB_page = db.Target_Pages
DB_Accounts = db.Accounts

In [3]:
def set_values(sender):
    global email_user
    global pass_user
    email_user=u.value
    pass_user=p.value

In [4]:
# u=widgets.Text(description='Usuario:')
# p=widgets.Password(description='Contraseña:')
# b=widgets.Button(description="Ingresar!")

# email_user=''
# pass_user=''

# display(u)
# display(p)
# display(b)



# u.on_submit(set_values)
# p.on_submit(set_values)
# b.on_click(set_values)

In [5]:
#Configuracion para poder encriptar y desencriptar password
def get_fenec():
    SECRET_KEY = b'SCRAP'
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32, salt=b"sAlT"*8, iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(SECRET_KEY))
    return Fernet(key)

In [6]:
#Buscamos en base de datos los datos de la cuenta de facebook y desencriptamos password
resulta=DB_Accounts.find_one({'code_name' : 'FB','state' : 1})

key = Fernet.generate_key()
f = get_fenec()
decrypted_pass =  f.decrypt(bytes(resulta['password'], encoding='utf-8')).decode()
email_user=resulta['user']
pass_user=decrypted_pass

In [7]:
#Celdas de seteo de variables

#Ruta del driver de Chrome o Firefox
diver_path="C:/Users/Usuario/Downloads/chromedriver_win32/chromedriver.exe"

#Order by para las publicaciones
order_post='?sorting_setting=CHRONOLOGICAL'
#order_post='?sorting_setting=TOP_POSTS'
#order_post='?sorting_setting=RECENT_ACTIVITY'

#Configuracion para deshabilitar "show notificacion"
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

driver=None
post_id_list=[]

In [8]:
#Obtengo las paginas
all_url_group=list(DB_page.find({}))
all_url_group

[{'_id': ObjectId('5ed4162eda6b430e309a3670'),
  'target_date_in': datetime.datetime(2020, 5, 31, 20, 40, 1, 461000),
  'target_link': 'https://www.facebook.com/groups/PilarRioSegundoyZona.CompraVenta/',
  'target_desc': 'Compra vende pilar Rio Segundo'},
 {'_id': ObjectId('5ed4167fda6b430e309a3671'),
  'target_date_in': datetime.datetime(2020, 5, 31, 20, 41, 24, 827000),
  'target_link': 'https://www.facebook.com/groups/1030505903629786/',
  'target_desc': 'Alquiler Rio Segundo Pilar'},
 {'_id': ObjectId('5ed417f6da6b430e309a3672'),
  'target_date_in': datetime.datetime(2020, 5, 31, 20, 47, 39, 892000),
  'target_link': 'https://www.facebook.com/groups/1582659365287516/',
  'target_desc': ' Alquileres de todo tipo en río II y zona'},
 {'_id': ObjectId('5ed4187ada6b430e309a3673'),
  'target_date_in': datetime.datetime(2020, 5, 31, 20, 47, 52, 980000),
  'target_link': 'https://www.facebook.com/groups/alquileresruta9/',
  'target_desc': 'Alquileres ruta 9'},
 {'_id': ObjectId('5ed418acd

In [9]:
def openBrowser():
    global driver
    driver = webdriver.Chrome(diver_path,chrome_options=chrome_options)
    driver.get('https://www.facebook.com/login.php?login_attempt=1&lwv=110')
    print("Facebook Abierto...")
    email = driver.find_element_by_xpath("//input[@id='email' or @name='email']")
    email.send_keys(email_user)
    print("Email ingresado...")
    password = driver.find_element_by_xpath("//input[@id='pass']")
    password.send_keys(pass_user)
    print("Password ingresado...")
    button = driver.find_element_by_xpath("//button[@id='loginbutton']")
    button.click()
    print("Facebook abierto")
    sleep(0.5)

In [10]:
def readAllPosts(url_group):
    print('Yendo al grupo')
    driver.get(url_group+order_post)

    for itera in range(200):
         #game.send_keys(Keys.SPACE)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5)

    # commentaires = driver.find_elements_by_xpath("//div [@data-testid='UFI2Comment/body']")
    #post_id = driver.find_elements_by_xpath("//div [@data-testid='story-subtitle']")
    #post_dates = driver.find_elements_by_xpath("//div [@class='_5ptz']")
    post_id = driver.find_elements_by_class_name("_5pcq")

    print('Cantidad de Post :', str(len(post_id)))

    global post_id_list

    for p in post_id:
        #print(com.text)
        post_id_list.append(p.get_attribute('href'))

In [11]:
def commentsCatch():
    # driver = webdriver.Chrome(diver_path)
    post_id=''
    date_post=''
    user_link=''
    user_photo_url=''
    user_name=''
    msg_post=''
    post_comment=''
    user_name_comment=''
    user_link_comment=''
    user_photo_url_comment=''
    comment_id=''
    date_comment=''
    com_exist=0
    
    global post_id_list

    for pid in post_id_list:

        data_post={}
        comments_post=[]

        #driver.get(url_group+'permalink/'+pid+'/')
        driver.get(pid)
        sleep(0.5)

        #Expandir respuestas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        #click_answer=driver.find_elements_by_xpath("//div [@class='_4swz _1i3s _293g']")
        click_answer=driver.find_elements_by_xpath("//div [@class='_4swz _77bj _1i3s _293g']")
        
        print(len(click_answer))
        
        if len(click_answer)>0:
            for c in click_answer:
                try:
                    c.click()
                except Exception as e:
                    print('Error: ',e)
                    pass
                sleep(0.5)

        post_dates = driver.find_elements_by_class_name("_5ptz")
        #post_user=driver.find_elements_by_class_name("_5pb8 e_r160ugzvu _8o _8s lfloat _ohe") 
        post_user = driver.find_elements_by_xpath("//a[@class='_5pb8 e_r160ugzvu _8o _8s lfloat _ohe']")
        post_user2 = driver.find_elements_by_xpath("//span[@class='fwb fcg']/a")
        div_photo = driver.find_elements_by_xpath("//div[@class='_38vo']")
        #print(post_user)
        post_msg = driver.find_elements_by_xpath("//div [@ data-testid='post_message']")
        post_ids = driver.find_elements_by_xpath("//input [@ name='ft_ent_identifier']")
        
        if len(div_photo)>0:
            user_photo_url=div_photo[0].find_element_by_tag_name("img").get_attribute('src')
        else:
            user_photo_url=''
        
                                                   
        if len(post_dates)>0:
            timestamP=int(post_dates[0].get_attribute('data-utime'))
            tP=datetime.fromtimestamp(timestamP)
            date_post=datetime.strptime(tP.strftime('%d/%m/%y %H:%M:%S'),'%d/%m/%y %H:%M:%S')
        else:
            date_post=''

        if len(post_user)>0:
            user_link=post_user[0].get_attribute('href')            
            user_name=post_user[0].get_attribute('title')
        else:
            # Si entra por aca significa que es un grupo
            if len(post_user2)>0:
                user_link=post_user2[0].get_attribute('href')            
                user_name=post_user2[0].get_attribute('title')
            else:
                user_link=''
                user_name=''

        if len(post_msg)>0:
            msg_post=post_msg[0].text
        else:
            msg_post=''

        if len(post_ids)>0:
            post_id=post_ids[0].get_attribute('value')
        else:
            post_id=''


        ##leemos comentarios
        #comments = driver.find_elements_by_class_name("_4eek clearfix _4eez _7793 clearfix")
        #comments = driver.find_elements_by_xpath("//div [@ data-testid='UFI2Comment/root_depth_0']")
        comments = driver.find_elements_by_xpath("//div [@ role='article']")
        #print(comments)

        if len(comments)>0:
            for idx,com in enumerate(comments):
                try:
                    #Primero obtengo el id limpio del comentario
                    comment_id=com.get_attribute('data-ft')
                    comment_id=comment_id.replace('{"tn":"R","ct":"','')
                    comment_id=comment_id.replace('="}','')

                    #Ahora verifico si ya no existe para no volverlo a insertar
                    result=DB_post.find_one({
                        'comments_post.comment_id': {
                            '$regex': '.*'+comment_id+'.*'}})
                    #print(result)
                    
                    #Verifico que el comentario no se haya leido dos veces en la pagina
                    for com in comments_post:
                        com_exist=0
                        if com['comment_id']==comment_id:
                            com_exist+=1

                    if result==None and com_exist==0: 
                        com1=com.find_element_by_class_name("_72vr")
                        com2=com.find_element_by_class_name("_6qw7")

                        post_comment=com1.find_element_by_xpath("./span[@dir='ltr']").text
                        user_name_comment=com1.find_element_by_xpath("./a[@class='_6qw4']").text
                        user_link_comment=com1.find_element_by_xpath("./a[@class='_6qw4']").get_attribute('href')
                        user_photo_url_comment=com.find_element_by_tag_name("img").get_attribute('src')

                        timestamp=int(com2.find_element_by_xpath("./abbr[@class='livetimestamp']").get_attribute('data-utime'))
                        t=datetime.fromtimestamp(timestamp)
                        date_comment=datetime.strptime(t.strftime('%d/%m/%y %H:%M:%S'),'%d/%m/%y %H:%M:%S')

                        #comments_post.append([comment_id,user_name_comment,user_link_comment,post_comment,date_comment])
                        comments_post.append({
                            'comment_id':comment_id,
                            'user_name_comment':user_name_comment,
                            'user_link_comment':user_link_comment,
                            'user_photo_url_comment': user_photo_url_comment,
                            'post_comment':post_comment,
                            'date_comment':date_comment,
                            'type' : 'reply',
                            'potable': isPotable(post_comment)
                    })
                    else:
                        print('Existe')
                except Exception as e2:
                    print('Error:', e2)
                    pass

        #data_post.append([pid,post_id,date_post,user_link,user_name,msg_post,comments_post])

        #Ahora verifico si ya no existe el post para solo actualizarle los comentarios
        result2=DB_post.find_one({
                        'post_id': {
                            '$regex': '.*'+post_id+'.*'}})

        if result2==None:
            data_post={
                'post_id':post_id,
                'post_link':pid,
                'date_post':date_post,
                'user_link':user_link,
                'user_photo_url':user_photo_url,
                'user_name':user_name,
                'msg_post':msg_post,
                'type' : 'post',
                'comments_post':comments_post,
                'potable': isPotable(msg_post)
            }
            id_=DB_post.insert_one(data_post).inserted_id
            print(id_)
        else:
            #Se insertan los comentarios que no existan
            for com in comments_post:
                DB_post.update_one({"post_id" : post_id},
                               {"$push": {'comments_post':com}}, True);

        sleep(0.5)

In [12]:
def isPotable(textPre):
    predict=model.predict(textPre.replace('\n',' ').lower())
    pred=int(predict[0][0].replace('__label__',''))
    return pred

In [13]:
#Cargo el modelo entrenado
modelPath='D:\Alexis\Proyectos\Text Classification (fastText)\model_inmobiliara.ftz'
model = fastText.load_model(modelPath)

In [14]:
def executeScrap():
    #1 - Abrimos el navegador
    openBrowser()

    #2 - Recorremos todas las paginas
    for url in all_url_group:
        url_group=url['target_link']

        #3 - Leemos todas las publicaciones de la pagina
        readAllPosts(url_group)

        #Obtenemos todos los comentarios
        commentsCatch()


In [ ]:
executeScrap()

C:\Users\Usuario\Miniconda3\envs\DataScience\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


Facebook Abierto...
Email ingresado...
Password ingresado...
Facebook abierto
Yendo al grupo


In [ ]:
#Solo para probar con una sola pagina en caso de querer evitar el loop por cada pagina
# post_id_list=['https://www.facebook.com/groups/alquileresruta9/permalink/519251155414651/']
# commentsCatch()

In [ ]:
# f = get_fenec()
# token = f.encrypt(bytes('Ragnarok15', encoding='utf-8'))
# print (token)